<a href="https://colab.research.google.com/github/AhmedZaghl0ul/Machine-Learning/blob/main/RNN_With_bbc_DataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import csv
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, LSTM,Embedding, Activation, Bidirectional, SimpleRNN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
vocab_size = 5000
emb_dim = 64
max_length = 200
oov_tok = '<OOV>'
training_potion = 0.8

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
articles = []
labels = []

with open("/content/drive/MyDrive/DataSets/bbc-text.csv") as csvfile:
  reader = csv.reader(csvfile, delimiter=',')
  next(reader)
  for row in reader:
    labels.append(row[0])
    aricle = row[1]
    for word in STOPWORDS:
      token = ' ' + word + ' '
      aricle = aricle.replace(token, ' ')
    articles.append(row[1])

In [6]:
len(labels) , len(articles)

(2225, 2225)

In [7]:
labels[0]

'tech'

In [8]:
unique_labels = set(labels)

In [9]:
print(unique_labels)

{'entertainment', 'business', 'sport', 'politics', 'tech'}


In [10]:
articles[0]

'tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being built-in to high

In [11]:
train_size = int(len(articles) * training_potion)

train_articles = articles[0: train_size]
train_labels = labels[0: train_size]

validation_articles = articles[train_size:]
validation_labels = labels[train_size:]

In [13]:
print("train_articles : " , len(train_articles))
print("train_labels : " , len(train_labels))
print("validation_articles : " , len(validation_articles))
print("validation_labels : " , len(validation_labels))

train_articles :  1780
train_labels :  1780
validation_articles :  445
validation_labels :  445


In [14]:
tokenizer = Tokenizer(num_words=vocab_size , oov_token=oov_tok)
tokenizer.fit_on_texts(train_articles)
word_index = tokenizer.word_index

In [15]:
len(word_index)

27299

In [17]:
word_index

{'<OOV>': 1,
 'the': 2,
 'to': 3,
 'of': 4,
 'and': 5,
 'a': 6,
 'in': 7,
 's': 8,
 'for': 9,
 'is': 10,
 'that': 11,
 'it': 12,
 'on': 13,
 'said': 14,
 'was': 15,
 'he': 16,
 'be': 17,
 'with': 18,
 'as': 19,
 'has': 20,
 'have': 21,
 'at': 22,
 'by': 23,
 'will': 24,
 'but': 25,
 'are': 26,
 'i': 27,
 'not': 28,
 'from': 29,
 'they': 30,
 'his': 31,
 'mr': 32,
 'we': 33,
 'this': 34,
 'an': 35,
 'had': 36,
 'which': 37,
 'would': 38,
 'been': 39,
 'their': 40,
 'year': 41,
 'more': 42,
 'up': 43,
 'its': 44,
 'also': 45,
 'were': 46,
 'who': 47,
 'people': 48,
 'new': 49,
 'us': 50,
 'there': 51,
 'one': 52,
 'about': 53,
 'after': 54,
 'out': 55,
 'can': 56,
 'or': 57,
 'than': 58,
 'all': 59,
 'if': 60,
 'over': 61,
 'could': 62,
 'you': 63,
 'last': 64,
 't': 65,
 'first': 66,
 'when': 67,
 'time': 68,
 'two': 69,
 'government': 70,
 'world': 71,
 'into': 72,
 'so': 73,
 'now': 74,
 'what': 75,
 'other': 76,
 'some': 77,
 'uk': 78,
 'she': 79,
 'best': 80,
 'years': 81,
 'against

In [19]:
tokenizer.texts_to_sequences(["the cat sat on my table"])

[[2, 1, 1, 13, 116, 2296]]

In [20]:
train_sequences = tokenizer.texts_to_sequences(train_articles)
len(train_sequences)

1780

In [21]:
train_sequences[0]

[183,
 259,
 7,
 2,
 1252,
 4,
 1216,
 18,
 129,
 1088,
 867,
 1,
 181,
 1412,
 4373,
 5,
 228,
 275,
 3743,
 1324,
 72,
 2,
 1305,
 1680,
 2,
 120,
 48,
 1002,
 183,
 24,
 17,
 1,
 442,
 7,
 176,
 81,
 68,
 11,
 10,
 229,
 3,
 35,
 3355,
 1325,
 37,
 2517,
 22,
 2,
 685,
 565,
 1485,
 142,
 7,
 3469,
 3612,
 3,
 1762,
 136,
 173,
 49,
 1030,
 24,
 841,
 52,
 4,
 125,
 952,
 1,
 18,
 2,
 50,
 712,
 2,
 1681,
 1217,
 5,
 76,
 506,
 24,
 17,
 2041,
 3,
 1216,
 842,
 129,
 652,
 179,
 1507,
 2111,
 1725,
 233,
 5,
 351,
 209,
 2451,
 3,
 905,
 1,
 5,
 1089,
 698,
 52,
 4,
 2,
 112,
 4043,
 53,
 1030,
 4,
 2657,
 20,
 39,
 228,
 5,
 452,
 275,
 3743,
 1,
 5,
 1,
 173,
 117,
 147,
 2962,
 97,
 2,
 50,
 8,
 4804,
 5,
 2,
 78,
 8,
 1413,
 235,
 528,
 48,
 3,
 242,
 1532,
 157,
 4586,
 5,
 548,
 1,
 183,
 1217,
 67,
 30,
 165,
 1,
 2,
 172,
 2112,
 9,
 135,
 42,
 1,
 183,
 30,
 26,
 45,
 89,
 1117,
 7,
 3,
 181,
 1412,
 183,
 2064,
 37,
 26,
 230,
 236,
 7,
 532,
 5,
 2,
 50,
 25,
 2963,
 3,
 

In [22]:
train_padded = pad_sequences(train_sequences , maxlen=max_length)

In [24]:
len(train_sequences[0])

750

In [25]:
len(train_padded) , len(train_padded[0])

(1780, 200)

In [26]:
validation_sequences = tokenizer.texts_to_sequences(validation_articles)
validation_padded = pad_sequences(validation_sequences , maxlen=max_length)

In [27]:
from tkinter import Label
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()

trainign_labels_encoded = lb.fit_transform(train_labels)
validation_labels_encoded = lb.fit_transform(validation_labels)

In [28]:
model = Sequential()

model.add(Embedding(vocab_size , emb_dim , input_length=len(train_sequences)))
model.add(Dropout(0.2))

model.add(SimpleRNN(units=emb_dim , return_sequences=True))
model.add(Dropout(0.2))

model.add(SimpleRNN(units=emb_dim))
model.add(Dropout(0.2))

model.add(Dense(64 , activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(5 , activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [29]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [30]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001 , decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy' , optimizer=opt , metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [31]:
num_epochs = 20
history = model.fit(train_padded ,
                    trainign_labels_encoded ,
                    epochs=num_epochs,
                    validation_data=(validation_padded , validation_labels_encoded),
                    verbose=2
                    )

Epoch 1/20
56/56 - 13s - 225ms/step - accuracy: 0.2163 - loss: 1.6507 - val_accuracy: 0.2360 - val_loss: 1.6021
Epoch 2/20
56/56 - 7s - 121ms/step - accuracy: 0.2343 - loss: 1.6144 - val_accuracy: 0.2899 - val_loss: 1.6010
Epoch 3/20
56/56 - 5s - 92ms/step - accuracy: 0.2882 - loss: 1.5631 - val_accuracy: 0.2899 - val_loss: 1.5760
Epoch 4/20
56/56 - 11s - 192ms/step - accuracy: 0.5022 - loss: 1.1943 - val_accuracy: 0.4876 - val_loss: 1.2514
Epoch 5/20
56/56 - 11s - 202ms/step - accuracy: 0.7315 - loss: 0.7716 - val_accuracy: 0.4809 - val_loss: 1.3434
Epoch 6/20
56/56 - 11s - 204ms/step - accuracy: 0.8978 - loss: 0.3383 - val_accuracy: 0.5034 - val_loss: 1.4278
Epoch 7/20
56/56 - 7s - 132ms/step - accuracy: 0.9725 - loss: 0.1062 - val_accuracy: 0.5371 - val_loss: 1.5483
Epoch 8/20
56/56 - 7s - 126ms/step - accuracy: 0.9803 - loss: 0.0769 - val_accuracy: 0.5281 - val_loss: 1.9867
Epoch 9/20
56/56 - 8s - 151ms/step - accuracy: 0.9837 - loss: 0.0554 - val_accuracy: 0.5528 - val_loss: 1.766